☝️
“LLMs are better at discriminating between options. Therefore, evaluations should focus on tasks like pairwise comparisons, classification, or scoring against specific criteria instead of open-ended generation. Aligning evaluation methods with LLMs' strengths in comparison leads to more reliable assessments of LLM outputs or model comparisons.”

### How can I measure the price of each openai API call (including embeddings)?  
*look into openai evals library: https://platform.openai.com/docs/guides/evals?api-mode=responses*
### Sources
https://platform.openai.com/docs/api-reference/evals  
https://cookbook.openai.com/examples/question_answering_using_embeddings  
https://cookbook.openai.com/examples/evaluation/use-cases/regression  
https://platform.openai.com/docs/guides/evals-design?api-mode=responses  
https://platform.openai.com/docs/guides/graders  

In [9]:
import os
import requests

# get the API key from environment
api_key = os.environ["OPENAI_API_KEY"]
headers = {"Authorization": f"Bearer {api_key}"}

# define a dummy grader for illustration purposes
grader = {
   "type": "score_model",
   "name": "my_score_model",
   "input": [
        {
            "role": "system",
            "content": "You are a text analytics, who evaluates relevance of an answer to a given question. You will be given a question and an answer, the goal is to deeply analyse it and come up with a score from 0 to 1, where 0 means the answer is completely irrelevant, and 1 - is the answer is absolutely relevant."
        },
        {
            "role": "user",
            "content": "Here is the question and answer to evaluate: Question: {{ item.input }} Answer: {{ sample.output_text }}"
        }
   ],
   "pass_threshold": 0.5,
   "model": "o4-mini-2025-04-16",
   "range": [0, 1],
   "sampling_params": {
       "max_completions_tokens": 32768,
       "top_p": 1,
       "reasoning_effort": "medium"
   },
}

# validate the grader
payload = {"grader": grader}
response = requests.post(
    "https://api.openai.com/v1/fine_tuning/alpha/graders/validate",
    json=payload,
    headers=headers
)
# print("validate response:", response.text)

# run the grader with a test reference and sample
payload = {
  "grader": grader,
  "item": {
     "input": "I see a charge on my credit card for $2,500 to \"ElectroMax\" on 2025-08-01 that I don't recognize. The card is 4111 1111 1111 1111. Please investigate and take action."
  },
  "model_sample": "Immediate response (sensitive action): 1) Confirm identity: ask customer to verify full name, date of birth (30), and last 4 of account (6789) or last 4 of card (1111). 2) While verifying, place an immediate temporary block on card number 4111 1111 1111 1111 to prevent further charges. 3) Create a fraud case with severity: high; record: merchant \"ElectroMax\", amount $2,500, date 2025-08-01, transaction ID (if available). 4) Provisionally credit the $2,500 pending investigation (explain provisional credit policy and timeline). 5) Start dispute process: request client-supplied evidence (receipt, photos, merchant communication), and confirm whether customer wants a replacement card issued. 6) Notify customer via their preferred contact_method (email: john.doe@example.com) and inform that two-factor authentication will be required for any account changes. 7) Next steps for customer: respond with a copy/photo of the cardholder’s ID and a short written statement if prompted, confirm whether they made any recent authorized transactions or gave card to a family member. 8) Internal actions: escalate to fraud team, monitor account 123456789 and routing 987654321 for suspicious transfers, log event with timestamp, agent ID, and actions taken. Estimated resolution time: 7–10 business days for full investigation; provisional credit may take 3–5 business days. If urgent, provide phone support option to +1-202-555-0199."
}
response = requests.post(
    "https://api.openai.com/v1/fine_tuning/alpha/graders/run",
    json=payload,
    headers=headers
)

print("run response:", response.text)

run response: {
  "reward": 1.0,
  "metadata": {
    "name": "my_score_model",
    "type": "score_model",
    "errors": {
      "formula_parse_error": false,
      "sample_parse_error": false,
      "sample_parse_error_details": null,
      "truncated_observation_error": false,
      "unresponsive_reward_error": false,
      "invalid_variable_error": false,
      "invalid_variable_error_details": null,
      "other_error": false,
      "python_grader_server_error": false,
      "python_grader_server_error_type": null,
      "python_grader_runtime_error": false,
      "python_grader_runtime_error_details": null,
      "model_grader_server_error": false,
      "model_grader_refusal_error": false,
      "model_grader_refusal_error_details": null,
      "model_grader_parse_error": false,
      "model_grader_parse_error_details": null,
      "model_grader_exceeded_max_tokens_error": false,
      "model_grader_server_error_details": null,
      "endpoint_grader_internal_error": false,
      